# This code is to get the labeled_energy_data csv file of each model

## Import the lib

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from Functions.LabelEnergyAnalysis import *

## load function

### interpolate function

In [14]:
@nb.jit(nopython=True)
def interpolate_point(times, powers, target_time):
    n = len(times)
    if n == 0:
        return 0.0
    if target_time <= times[0]:
        return powers[0]
    if target_time >= times[-1]:
        return powers[-1]
    
    # Binary search
    left, right = 0, n-1
    while left <= right:
        mid = (left + right) // 2
        if times[mid] == target_time:
            return powers[mid]
        elif times[mid] < target_time:
            left = mid + 1
        else:
            right = mid - 1
            
    # Linear interpolation
    pos = left
    t1, p1 = times[pos-1], powers[pos-1]
    t2, p2 = times[pos], powers[pos]
    ratio = (target_time - t1) / (t2 - t1)
    return p1 + (p2 - p1) * ratio

@nb.jit(nopython=True)
def integrate_power_over_interval(samples, start_time, end_time):
    times = samples[:, 0]
    powers = samples[:, 1]
    
    # Get start and end powers through interpolation
    start_power = interpolate_point(times, powers, start_time)
    end_power = interpolate_point(times, powers, end_time)
    
    # Filter points within interval
    mask = (times >= start_time) & (times <= end_time)
    interval_times = times[mask]
    interval_powers = powers[mask]
    
    # Create array including boundary points
    n_points = len(interval_times)
    full_times = np.zeros(n_points + 2)
    full_powers = np.zeros(n_points + 2)
    
    # Add boundary points
    full_times[0] = start_time
    full_powers[0] = start_power
    full_times[-1] = end_time
    full_powers[-1] = end_power
    
    # Add interior points
    if n_points > 0:
        full_times[1:-1] = interval_times
        full_powers[1:-1] = interval_powers
    
    # Integration using trapezoidal rule
    total_energy = 0.0
    for i in range(len(full_times)-1):
        dt = full_times[i+1] - full_times[i]
        avg_p = (full_powers[i] + full_powers[i+1]) / 2.0
        total_energy += avg_p * dt
        
    return total_energy


### label energy function

In [15]:
def label_energy_consumption(energy_data, to_device, forward, loss, backward, optimize):
    # Create a copy of the energy_data dataframe to avoid modifying the original
    labeled_energy_data = energy_data.copy()
    
    # Initialize a new column for the step labels
    labeled_energy_data['step'] = 'idle'
    
    # Define a helper function to label the steps
    def label_steps(energy_data, step_energy, step_name):
        for epoch in range(step_energy.shape[0]):
            for batch in range(step_energy.shape[1]):
                start_time = step_energy[epoch][batch][0]
                end_time = step_energy[epoch][batch][1]
                mask = (energy_data['timestamp'] >= start_time) & (energy_data['timestamp'] <= end_time)
                labeled_energy_data.loc[mask, 'step'] = step_name
    
    # Label each step
    label_steps(labeled_energy_data, to_device, 'to_device')
    label_steps(labeled_energy_data, forward, 'forward')
    label_steps(labeled_energy_data, loss, 'loss')
    label_steps(labeled_energy_data, backward, 'backward')
    label_steps(labeled_energy_data, optimize, 'optimize')
    
    return labeled_energy_data

### load the label function

## find the data path

In [16]:
current_path = os.getcwd()
data_path = os.path.join(current_path, 'ModelsData')
data_folders = os.listdir(data_path)
print(data_folders)

['googlenet_origin', 'resnet50', 'resnet34', 'mobilenetv2', 'vgg13', 'googlenet_mod7', 'googlenet_mod9', 'googlenet_mod8', 'googlenet_mod1', 'googlenet_mod6', 'resnet18', 'mobilenetv1', 'vgg16', 'vgg11', 'googlenet_mod3', 'googlenet_mod4', 'googlenet_mod5', 'googlenet_mod2', 'alexnet']


## load all the model data

In [17]:
# set the epoch number, batch size and the round number as well as the sampling rate
epoch = 5
batch_size = 256
round_num = 0
sampling_rate = 2

In [18]:
# for all the folder names in the data folder, generate the path to the folder
# and print the path
folder_name = f'E{epoch}_B{batch_size}_R{round_num}_SR{sampling_rate}'

data_dir = 'fashion_mnist'
# data_dir = 'cifar100'
# data_dir = 'cifar10'



print(folder_name)
for folder in data_folders:
    folder_path = os.path.join(data_path, folder)
    folder_path = os.path.join(folder_path, folder_name)
    folder_path = os.path.join(folder_path, data_dir)
    print(folder_path)

    # load the csv files 
    energy_data = pd.read_csv(os.path.join(folder_path, 'energy_consumption_file.csv'))
    energy_data['timestamp'] = pd.to_numeric(energy_data['timestamp'], errors='coerce')


    # load the npy files
    to_device = np.load(os.path.join(folder_path, 'to_device.npy'), allow_pickle=True)
    forward = np.load(os.path.join(folder_path, 'forward.npy'), allow_pickle=True)
    loss = np.load(os.path.join(folder_path, 'loss.npy'), allow_pickle=True)
    backward = np.load(os.path.join(folder_path, 'backward.npy'), allow_pickle=True)
    optimize = np.load(os.path.join(folder_path, 'optimize.npy'), allow_pickle=True)

    # Set the display format for floating-point numbers to avoid scientific notation
    pd.options.display.float_format = '{:.6f}'.format

    # Use the function to label the energy consumption data
    labeled_energy_data = label_energy_consumption(energy_data, to_device, forward, loss, backward, optimize)
    print(labeled_energy_data.head())

    # save the file to the folder
    labeled_energy_data.to_csv(os.path.join(folder_path, 'labeled_energy_data.csv'), index=False)

E5_B256_R0_SR2
/Users/dtjgp/Projects/GreenAI/3080/ModelsData/googlenet_origin/E5_B256_R0_SR2/fashion_mnist
          timestamp  power_in_watts  step
0 1734521196.030653       90.938000  idle
1 1734521196.262849       90.942000  idle
2 1734521196.267752       90.942000  idle
3 1734521196.535047       90.891000  idle
4 1734521196.843291       90.826000  idle
/Users/dtjgp/Projects/GreenAI/3080/ModelsData/resnet50/E5_B256_R0_SR2/fashion_mnist
          timestamp  power_in_watts  step
0 1734520758.850771       91.621000  idle
1 1734520759.019024       91.609000  idle
2 1734520759.181253       91.578000  idle
3 1734520759.348285       91.437000  idle
4 1734520759.512284       91.437000  idle
/Users/dtjgp/Projects/GreenAI/3080/ModelsData/resnet34/E5_B256_R0_SR2/fashion_mnist


/var/folders/d9/50r202r14gbc_shcxg960xrh0000gn/T/ipykernel_25239/1227709553.py:19: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  energy_data = pd.read_csv(os.path.join(folder_path, 'energy_consumption_file.csv'))


          timestamp power_in_watts  step
0 1734520246.312647          16.54  idle
1 1734520246.475491         23.233  idle
2 1734520246.480771         23.233  idle
3 1734520246.636172         40.216  idle
4 1734520246.756206         43.516  idle
/Users/dtjgp/Projects/GreenAI/3080/ModelsData/mobilenetv2/E5_B256_R0_SR2/fashion_mnist
          timestamp  power_in_watts  step
0 1734610932.664366       13.044000  idle
1 1734610932.846767       21.837000  idle
2 1734610932.852010       21.837000  idle
3 1734610933.032963       28.630000  idle
4 1734610933.211537       34.058000  idle
/Users/dtjgp/Projects/GreenAI/3080/ModelsData/vgg13/E5_B256_R0_SR2/fashion_mnist


/var/folders/d9/50r202r14gbc_shcxg960xrh0000gn/T/ipykernel_25239/1227709553.py:19: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  energy_data = pd.read_csv(os.path.join(folder_path, 'energy_consumption_file.csv'))


          timestamp power_in_watts  step
0 1734560268.578758         19.743  idle
1 1734560268.671264         30.368  idle
2 1734560268.747631         35.089  idle
3 1734560268.750994         35.089  idle
4 1734560268.818558         35.089  idle
/Users/dtjgp/Projects/GreenAI/3080/ModelsData/googlenet_mod7/E5_B256_R0_SR2/fashion_mnist
          timestamp  power_in_watts  step
0 1734523364.654312       89.225000  idle
1 1734523364.909820       89.259000  idle
2 1734523365.152238       89.263000  idle
3 1734523365.156598       89.263000  idle
4 1734523365.403755       89.258000  idle
/Users/dtjgp/Projects/GreenAI/3080/ModelsData/googlenet_mod9/E5_B256_R0_SR2/fashion_mnist
          timestamp  power_in_watts  step
0 1734524453.766207       90.558000  idle
1 1734524454.030741       90.341000  idle
2 1734524454.035138       90.341000  idle
3 1734524454.292768       90.300000  idle
4 1734524454.296117       90.300000  idle
/Users/dtjgp/Projects/GreenAI/3080/ModelsData/googlenet_mod8/E5_B256_R